<a href="https://colab.research.google.com/github/adityassharma-ss/FakeNews-Detection/blob/main/FakeNewsDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing The Dependencies


In [4]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data-Preprocessing

In [8]:
news_dataset = pd.read_csv('/content/train.csv')

In [9]:
news_dataset.shape

(20800, 5)

In [10]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [11]:
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [12]:
news_dataset = news_dataset.fillna('')

In [13]:
news_dataset['content'] =  news_dataset['author']+' '+news_dataset['title']

In [14]:
print(news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [15]:
X=news_dataset.drop(columns='label',axis=1)
Y=news_dataset['label']

In [16]:
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

Stemming: Process of reducing a word to its root word

In [17]:
port_stem = PorterStemmer()

In [18]:
def stemming(content):
  stemmed_content = re.sub('[˄a-zA-z]',' ', content)
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content


In [19]:
news_dataset['content'] =  news_dataset['content'].apply(stemming)

In [20]:
print(news_dataset['content'])

0              : ’ ’
1            . : , -
2                  .
3                 15
4                   
            ...     
20795       . .: ’ ’
20796    . . . : , -
20797        . ’ ’ -
20798              ,
20799            -35
Name: content, Length: 20800, dtype: object


In [21]:
# Separating the data and Label

X = news_dataset['content'].values
Y = news_dataset['label'].values

In [22]:
print(X)

[': ’ ’' '. : , -' '.' ... '. ’ ’ -' ',' '-35']


In [23]:
print(Y)

[1 0 1 ... 0 1 1]


In [24]:
Y.shape

(20800,)

In [25]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)


In [26]:
print(X)

  (3, 62)	1.0
  (27, 62)	1.0
  (31, 145)	0.4249057758414498
  (31, 20)	0.49454392483696136
  (31, 7)	0.7582093299765108
  (36, 146)	0.7333171625424529
  (36, 131)	0.6798867105045412
  (37, 220)	0.8630627144132533
  (37, 145)	0.5050967738855863
  (40, 131)	1.0
  (47, 1202)	0.3691999620908306
  (47, 1122)	0.38440673900713157
  (47, 1056)	0.3691999620908306
  (47, 1038)	0.3691999620908306
  (47, 1012)	0.38440673900713157
  (47, 937)	0.38440673900713157
  (47, 867)	0.38440673900713157
  (53, 1384)	1.0
  (56, 251)	1.0
  (61, 47)	1.0
  (73, 348)	1.0
  (75, 263)	1.0
  (76, 263)	0.7876361228177006
  (76, 1)	0.6161406803910127
  (78, 243)	1.0
  :	:
  (20680, 1)	0.6161406803910127
  (20690, 1095)	0.3159830601044173
  (20690, 1087)	0.3159830601044173
  (20690, 1077)	0.3159830601044173
  (20690, 891)	0.3159830601044173
  (20690, 798)	0.2244087729413534
  (20690, 782)	0.2244087729413534
  (20690, 779)	0.3159830601044173
  (20690, 732)	0.3034830609713406
  (20690, 704)	0.2663660348465984
  (20690, 6

Spltiing the data to training & set data

In [27]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, stratify = Y, random_state=2)

Training the Model : Logistic regression

In [28]:
model = LogisticRegression()

In [29]:
model.fit(X_train, Y_train)

LogisticRegression()

Evaluation

Accuracy Score

In [30]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [31]:
print('Accuracy score of the training data is: ' , training_data_accuracy)

Accuracy score of the training data is:  0.5408052884615384


In [32]:
#On test Data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [33]:
print('Accuracy score for the test data: ', test_data_accuracy)

Accuracy score for the test data:  0.5225961538461539


Make a Predictive System

In [36]:
X_new = X_test[3]

prediction = model.predict(X_new)
print(prediction)

if(prediction[0]==0):
  print('The news is Real')
else:
  print('Its a Fake news')

# You can fill indexing in test[ ] and check if the news is real or fake

[0]
The news is Real


In [37]:
print(Y_test[3])

0
